## Binary semantic segmentation example using U-Net
Preparation of dataset and model training code from here:

https://pyimagesearch.com/2021/11/08/u-net-training-image-segmentation-models-in-pytorch/

In [1]:
import os
import glob
import matplotlib.pyplot as plt

import torch
import torchvision
from tqdm import tqdm

print(torch.__version__)
print(torchvision.__version__)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

# determine if we will be pinning memory during data loading
PIN_MEMORY = True if DEVICE == "cuda" else False

1.13.1
0.14.1
cuda


/home/shymon/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### CONFIGURE PATHS AND HYPERPARAMETERS FOR TRAINING BELOW.

In [2]:
GD_PATH = os.getcwd() # get current working directory for the repo
print(GD_PATH)

# PROVIDE PATH TO DOWNLOADED MAPAI DATASET
DATASET_PATH = "/home/shymon/datasets/"

DATASET_PATH = os.path.join(DATASET_PATH, "mapai_full") # create dataset path

print(DATASET_PATH)

TRAIN_IMG_DIR = os.path.join(DATASET_PATH, "train", "images")
TRAIN_MASK_DIR = os.path.join(DATASET_PATH, "train", "masks")

print(TRAIN_IMG_DIR)
print(TRAIN_MASK_DIR)

VAL_IMG_DIR = os.path.join(DATASET_PATH, "validation", "images")
VAL_MASK_DIR = os.path.join(DATASET_PATH, "validation", "masks")

print(VAL_IMG_DIR)
print(VAL_MASK_DIR)

TEST_IMG_DIR = os.path.join(DATASET_PATH, "task1_test", "images")
TEST_MASK_DIR = os.path.join(DATASET_PATH, "task1_test", "masks")

print(TEST_IMG_DIR)
print(TEST_MASK_DIR)

# CONFIGURE MapAI DATASET
NUM_CHANNELS = 3
NUM_LEVELS  = 3
NUM_CLASSES = 1

# IMAGE SHAPE
IMG_WIDTH = 512
IMG_HEIGHT = 512

#---------------------------------------------------------------------------------------------------#

# CONFIGURE parameters for training
EPOCHS = 25
init_lr = 1e-4 # learning rate
BATCH_SIZE = 2

THRESHOLD  = 0.5
base_output = "out"

model_name = "unet-25-epochs.pth" # provide name for model
training_plot_name = "unet-25-epochs.png"

#---------------------------------------------------------------------------------------------------#

# OUTPUT PATHS

# Trained model path
MODEL_PATH = os.path.join(GD_PATH, "trained_models", model_name) # change depending on the number of epochs
print(MODEL_PATH)
PLOT_PATH  = os.path.join(GD_PATH, "plots", training_plot_name) # the folder to save future plots
print(PLOT_PATH)

/home/shymon/Documents/mapAI-regularization
/home/shymon/datasets/mapai_full
/home/shymon/datasets/mapai_full/train/images
/home/shymon/datasets/mapai_full/train/masks
/home/shymon/datasets/mapai_full/validation/images
/home/shymon/datasets/mapai_full/validation/masks
/home/shymon/datasets/mapai_full/task1_test/images
/home/shymon/datasets/mapai_full/task1_test/masks
/home/shymon/Documents/mapAI-regularization/trained_models/unet-25-epochs.pth
/home/shymon/Documents/mapAI-regularization/plots/unet-25-epochs.png


### Load and read the MapAI dataset

In [3]:
import tifffile
from torch.utils.data import Dataset
import cv2


class mapAIdataset(Dataset):
    def __init__(self, imagePaths, maskPaths, transforms):
        # store the image and mask filepaths, and augmentation
        # transforms
        self.imagePaths = imagePaths
        self.maskPaths = maskPaths
        self.transforms = transforms
        
    def __len__(self):
        # return the number of total samples contained in the dataset
        return len(self.imagePaths)
    
    def __getitem__(self, idx):
        # grab the image path from the current index
        imagePath = self.imagePaths[idx]
        # load the image from disk, swap its channels from BGR to RGB,
        # and read the associated mask from disk
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = tifffile.imread(self.maskPaths[idx])
        # convert the mask to a float32 tensor with values in the range [0, 1]
        mask = mask.astype('float32')
        # check to see if we are applying any transformations
        if self.transforms is not None:
            # apply the transformations to both image and its mask
            image = self.transforms(image)
            mask = self.transforms(mask)
    
        # return a tuple of the image and its mask
        return (image, mask)

### Build UNET architecture

In [4]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # DOWN part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # UP part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

### Training the segmentation model
Below we append the paths for TRAIN/VAL/TEST sets - images/masks.

In [5]:
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from imutils import paths
import time

# TRAINING
train_images = sorted(list(paths.list_images(TRAIN_IMG_DIR)))
train_masks = sorted(list(paths.list_images(TRAIN_MASK_DIR)))

# VALIDATION
val_images = sorted(list(paths.list_images(VAL_IMG_DIR)))
val_masks = sorted(list(paths.list_images(VAL_MASK_DIR)))


# TEST
test_images = sorted(list(paths.list_images(TEST_IMG_DIR)))
test_masks = sorted(list(paths.list_images(TEST_MASK_DIR)))

### Define transformations

I tried out different data augmentation techniques, including Horizontal Flip, Vertical Flip, Contrast, Brightness. They did not improve my results much, the validation and training loss were actually worse than without data augmentation techniques.

https://pytorch.org/vision/stable/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py

https://albumentations.ai/docs/getting_started/mask_augmentation/

In [6]:
import torchvision.transforms as T

# T.RandomHorizontalFlip(p=0.5),
# T.RandomVerticalFlip(p=0.1),

# Image augmentations applied
transforms = T.Compose([T.ToPILImage(),
                        T.Resize((IMG_HEIGHT,IMG_WIDTH)),
                        T.ToTensor()])

# create the train and test datasets
trainDS = mapAIdataset(imagePaths=train_images,
                       maskPaths=train_masks,
                       transforms=transforms)

valDS = mapAIdataset(imagePaths=val_images,
                     maskPaths=val_masks,
                     transforms=transforms)

testDS = mapAIdataset(imagePaths=test_images,
                      maskPaths=test_masks,
                      transforms=transforms)

print(f"[INFO] found {len(trainDS)} examples in the TRAINING set...")
print(f"[INFO] found {len(valDS)} examples in the VALIDATION set...")
print(f"[INFO] found {len(testDS)} examples in the TEST set...")

# create the training and test data loaders
trainLoader = DataLoader(trainDS,
                         shuffle=True,
                         batch_size=BATCH_SIZE,
                         pin_memory=PIN_MEMORY,
                         num_workers=os.cpu_count())

valLoader = DataLoader(valDS,
                       shuffle=False,
                       batch_size=BATCH_SIZE,
                       pin_memory=PIN_MEMORY,
                       num_workers=os.cpu_count())

testLoader = DataLoader(testDS,
                        shuffle=False,
                        batch_size=BATCH_SIZE,
                        pin_memory=PIN_MEMORY,
                        num_workers=os.cpu_count())

[INFO] found 6769 examples in the TRAINING set...
[INFO] found 1469 examples in the VALIDATION set...
[INFO] found 1368 examples in the TEST set...


### Initialize UNET model for training

Here we initialize the defined UNET model for training and calculate the steps per epoch for train/val/test set.

In [7]:
model = UNET().to(DEVICE)

# loss / optimizer
lossFunction = BCEWithLogitsLoss()
opt = Adam(model.parameters(), lr=init_lr, weight_decay=0.001)

# calculate steps per epoch for train/val/test
trainSteps = len(trainDS) // BATCH_SIZE 
valSteps = len(valDS) // BATCH_SIZE
testSteps = len(testDS) // BATCH_SIZE

print(trainSteps, valSteps, testSteps)

# initialize a dictionary to store training history
H = {"train_loss": [], "val_loss": [], "test_loss": []}
H

3384 734 684


{'train_loss': [], 'val_loss': [], 'test_loss': []}

In [8]:
torch.cuda.empty_cache() # PyTorch thing to empty cache

### TRAINING THE MODEL

Run this piece of code only if you want to train the model from scratch.

Training locally: BATCH_SIZE  = 2 takes 5035 MB of GPU memory.



In [ ]:
# loop over epochs
print("[INFO] training UNET ...")
startTime = time.time()

for epoch in tqdm(range(EPOCHS)):
    model.train()

    # initialize total training and validation loss
    totalTrainLoss = 0
    totalValLoss = 0
    totalTrainAcc = 0
    totalValAcc = 0

    # loop over the training set
    for (i, (x, y)) in enumerate(trainLoader):
        # send output to device
        (x, y) = (x.to(DEVICE), y.to(DEVICE))

        # perform a forward pass and calculate the training loss
        pred = model(x)
        loss = lossFunction(pred, y)

        # calculate the accuracy
        acc = ((pred > 0.5) == y).float().mean()

        # kill previously accumulated gradients then
        # perform backpropagation and update model parameters
        opt.zero_grad()
        loss.backward()
        opt.step()

        # add the loss and accuracy to the total training loss and accuracy
        totalTrainLoss += loss
        totalTrainAcc += acc

    # switch of autograd
    with torch.no_grad():
        # set the model in evaluation mode
        model.eval()

        # loop over the validation set
        for (x, y) in valLoader:
            # send the input to the device
            (x, y) = (x.to(DEVICE), y.to(DEVICE))

            # make the predictions and calculate the validation loss
            pred = model(x)
            loss = lossFunction(pred, y)

            # calculate the accuracy
            acc = ((pred > 0.5) == y).float().mean()

            # add the loss and accuracy to the total validation loss and accuracy
            totalValLoss += loss
            totalValAcc += acc

    # calculate the average training and validation loss and accuracy
    avgTrainLoss = totalTrainLoss / trainSteps
    avgValLoss = totalValLoss / valSteps
    avgTrainAcc = totalTrainAcc / trainSteps
    avgValAcc = totalValAcc / valSteps
        
    # update our training history
    H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
    H["val_loss"].append(avgValLoss.cpu().detach().numpy())

    # print the model training and validation information
    print("[INFO] EPOCH: {}/{}".format(epoch + 1, EPOCHS))
    print("Train loss: {:.6f}, Train acc: {:.6f}, Val loss: {:.4f}, Val acc: {:.4f}".format(
        avgTrainLoss, avgTrainAcc, avgValLoss, avgValAcc))
        
# display the total time needed to perform the training
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(endTime - startTime))

Train loss: 0.001194, Val loss: 0.0013
[INFO] total time taken to train the model: 27115.38s

In [ ]:
H # show traning/val loss history

#### Training times

*   5 epochs:  1068.83s
*   10 epochs: 1926.91s
*   50 epochs: 9461.90s = 2h 37m 41s  
*   100 epochs: 20829 s  = 5 h 45 m
*   100 epochs - Premium GPU: 2314.83s = 39 min

The mentioned models are stored on my computer and on Google Drive.


### Plot the training and validation loss

In [ ]:
PLOT_PATH

In [ ]:
# plot the training loss
print(MODEL_PATH)
print(PLOT_PATH)

plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["val_loss"], label="val_loss")
plt.title("Training Loss on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig(PLOT_PATH)
# serialize the model to disk
torch.save(model, MODEL_PATH) # saves the model

As you can see the validation loss is jumping up and down. I do not know why this happens.

### Prediction part

Here the trained model is loaded and use for prediction on test images.

In [9]:
# Load saved model for prediction

print(MODEL_PATH)

model = torch.load(MODEL_PATH) # add MODEL_PATH after training
print("model loaded for prediction")

model

/home/shymon/Documents/mapAI-regularization/trained_models/unet-25-epochs.pth
model loaded for prediction


UNET(
  (ups): ModuleList(
    (0): ConvTranspose2d(1024, 512, kernel_size=(2, 2), stride=(2, 2))
    (1): DoubleConv(
      (conv): Sequential(
        (0): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (2): ConvTranspose2d(512, 256, kernel_size=(2, 2), stride=(2, 2))
    (3): DoubleConv(
      (conv): Sequential(
        (0): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), p

#### Provide test images for MapAI Dataset

In [10]:
PREDICTIONS_DIR = os.path.join(GD_PATH, "predictions")
PREDICTIONS_DIR

'/home/shymon/Documents/mapAI-regularization/predictions'

#### Make predictions on the entire MapAI dataset

Make predictions on test images and save them to the folder named predictions.

In [ ]:
import random
import gc
from pathlib import Path
import numpy as np
from PIL import Image

# PLOTTING PREDICTIONS AS SINGLE IMAGES

# Output folder for the predictions
output_folder = PREDICTIONS_DIR + "/" # check for Windows to save predictions inside the folder

# PLOT TEST IMAGES as RGB
for n in range(len(test_images)):
  gc.collect()
  # Test image number
  testImgName = str(Path(test_images[n]).stem) + '.tif'
  #print('#', testImgName)

   # Make predicton on a test image specified with counter n
  test_img = test_images[n]
  test_img_input = np.expand_dims(test_img, 0)
  #print('#', test_img_input[0])

  # PyTorch --> works
  model.eval()
  with torch.no_grad():
    image = cv2.imread(test_img_input[0])
    image = cv2.resize(image, dsize = (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_CUBIC)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype("float32") / 255
    
    # print('SIZE: ', image.shape)

    # make the channel axis to be the leading one, add batch dimension
    image = np.transpose(image, (2, 0, 1))
    # create a PyTorch tensor
    image = np.expand_dims(image, 0)
    # flash the tensor to the device
    image  = torch.from_numpy(image).to(DEVICE)

    # make the prediction
    predMask = model(image).squeeze()
    # pass result through sigmoid
    predMask = torch.sigmoid(predMask)

    # convert result to numpy array
    predMask = predMask.cpu().numpy()

    # filter out the weak predictions and convert them to integers
    predMask = (predMask > THRESHOLD) * 255
    predMask = predMask.astype(np.uint8)

    # generate image from array
    pIMG = Image.fromarray(predMask)
    pIMG.save(str(output_folder + testImgName))

    print('Prediction:', testImgName, 'saved to:', output_folder)

#### Make predictions on single images by choice

Change the parameter n to choose which image to plot.

In [ ]:
# ----------------------------------------------------------------------

predictions = glob.glob(output_folder + "*.tif")
predictions.sort()
print("# IMAGES for prediction: ", len(predictions))
print("Choosen n can be from 0 o 1367! ")

# ----------------------------------------------------------------------

n = 900 # change this number depending on which image you want to test

fig = plt.figure(figsize=(18,12))
ax1 = fig.add_subplot(131)

ax1.set_title('RGB image: ')
image = cv2.imread(test_images[n])[:,:,::-1]
ax1.imshow(image)
ax1.set_axis_off()

ax2 = fig.add_subplot(132)
ax2.set_title('Ground truth: ')
image = cv2.imread(test_masks[n])[:,:,::-1]
image *= 255
ax2.imshow(image)
ax2.set_axis_off()

ax3 = fig.add_subplot(133)
ax3.set_title('Prediction: ')
image = cv2.imread(predictions[n])[:,:,::-1]
ax3.imshow(image)
ax3.set_axis_off()

### BUILDING FOOTPRINT REGULARIZATION

Used repo: https://github.com/zorzi-s/projectRegularization

git clone the repo to the folder where your notebook is stored. To get curent working directory use os.getcwd().

The pretrained weights need to be downloaded from the provided link and saved into the folder pretrained_weighs that is inside projectRegularization:

https://drive.google.com/drive/folders/1IPrDpvFq9ODW7UtPAJR_T-gGzxDat_uu

Next step is to generate a Python file to locate the necessary pretrained weights from projectRegularization. The code below was only tested on Ubuntu, not on Windows.

In [ ]:
# DEFINE NECESSARY PATHS FOR REGULARIZATION PART

projectRegDir = os.path.join(GD_PATH, "projectRegularization")
print(projectRegDir)

ptw = os.path.join(projectRegDir, "pretrained_weights") 
print(ptw)

# OUTPUT REGULARIZATIONS DIR
REGULARIZATION_DIR = os.path.join(GD_PATH, "regularizations") + "/"
print(REGULARIZATION_DIR)

# GET THE PATHS FOR TRAINED GAN MODELS
ENCODER = os.path.join(ptw, "E140000_e1")
GENERATOR = os.path.join(ptw, "E140000_net")

print(ENCODER)
print(GENERATOR)

In [ ]:
# CREATE A NEW variables.py WITH USERS PATHS

with open(projectRegDir + 'variables.py', 'w') as f:
    f.write('# CONFIGURE THE PATHS HERE: \n\n')
    f.write('# TRAINING \n')
    f.write('DATASET_RGB = ' + '"' + str(TRAIN_IMG_DIR + '*.tif' + '"') + '\n')
    f.write('DATASET_GTI = ' + '"' + str(TRAIN_MASK_DIR + '*.tif' + '"') + '\n')
    f.write('DATASET_SEG = ' + '"' + str(PREDICTIONS_DIR + '*.tif' + '"') + '\n')
    f.write('\n')
    f.write('DEBUG_DIR = ' + '"' + str('./debug/') + '"' + '\n')
    f.write('\n')
    f.write('# INFERENCE \n')
    f.write('INF_RGB = ' + '"' + str(TEST_IMG_DIR + '*.tif' + '"') + '\n')
    f.write('INF_SEG = ' + '"' + str(PREDICTIONS_DIR + '*.tif' + '"') + '\n')
    f.write('INF_OUT = ' + '"' + str(REGULARIZATION_DIR + '"') + '\n')
    f.write('\n')
    f.write('MODEL_ENCODER = ' + '"' + str(ENCODER) + '"' + '\n')
    f.write('MODEL_GENERATOR = ' + '"' + str(GENERATOR) + '"' + '\n')
    f.close()
 
print("variables.py created with users paths...")


#### Run projectRegularization

Takes around 6-8 minutes.

You only need to change the command below and replace it with the absolute path for regularize.py


In [ ]:
# RUN REGULARIZATION - takes around 6-8 minutes
# change the command below: provide the absolute path for regularize.py

!python /home/shymon/Documents/mapAI-regularization/projectRegularization/regularize.py

### Compare predictions and regularizations on a single image

In [ ]:
# Read Regularizations to plot and compare results

regularizations = glob.glob(REGULARIZATION_DIR + "*.tif")
regularizations.sort()

print("# of predicted images: ", len(predictions))
print("# of regularized images: ", len(regularizations))

Code to plot RGB, GT, PREDICTION and REGULARIZATION in a single plot for comparison.

Change parameter n accordingly.

In [ ]:
n = 600

fig = plt.figure(figsize=(18,12))
ax1 = fig.add_subplot(141)

ax1.set_title('RGB: ')
image = cv2.imread(test_images[n])[:,:,::-1]
ax1.imshow(image)
ax1.set_axis_off()

ax2 = fig.add_subplot(142)
ax2.set_title('Ground truth: ')
image = cv2.imread(test_masks[n])[:,:,::-1]
image *= 255
ax2.imshow(image)
ax2.set_axis_off()

ax3 = fig.add_subplot(143)
ax3.set_title('Prediction: ')
image = cv2.imread(predictions[n])[:,:,::-1]
ax3.imshow(image)
ax3.set_axis_off()

ax4 = fig.add_subplot(144)
ax4.set_title('Regularization: ')
image = cv2.imread(regularizations[n])[:,:,::-1]
ax4.imshow(image)
ax4.set_axis_off()

# DEFINE PATH FOR PLOTS TO BE SAVED
figPath = GD_PATH + "/" + "plots" + "/" "compare-" + str(n) + ".png"
print(figPath)

# Save plot
fig.savefig(figPath)

### VECTORIZING THE REGULARIZED BUILDING MASKS with GDAL

GDAL: https://gdal.org/'

GDAL: https://www.youtube.com/watch?v=q3DLdMj5zLA

I do not know if it is possible to install GDAL on WINDOWS inside a conda environment.

On Ubuntu you have to follow these steps:



Specific process for installation: https://stackoverflow.com/questions/44005694/no-module-named-gdal

- sudo apt-get update && sudo apt upgrade -y && sudo apt autoremove 
- sudo apt-get install -y cdo nco gdal-bin libgdal-dev-
- python -m pip install --upgrade pip setuptools wheel
- python -m pip install --upgrade gdal
- conda install -c conda forge libgdal
- conda install -c conda-forge libgdal
- conda install -c conda-forge gdal
- conda install tiledb=2.2
- conda install poppler

When you have this you can hopefully vectorize the detected masks quite easily.

In [ ]:
def get_fname_from_path(path):
    """
    Given a path, returns the filename after the last frontslash character.
    """
    return path.rsplit('/', 1)[-1]

def get_fname_no_extension(path):
    """
    Given a path, returns the filename without its extension.
    """
    filename, extension = os.path.splitext(path)
    return filename

In [ ]:
import osgeo
from osgeo import gdal
from osgeo import ogr
print('GDAL version: ', osgeo.gdal.__version__)

# Choose which image to vectorize
n  = 0

input = regularizations[n]
print()
print("INPUT: ", input)

# print(get_fname_no_extension(input))

# out
output = get_fname_from_path(get_fname_no_extension(input)) + ".gpkg"
print("OUTPUT: ", output)

# Open image with GDAl driver
ds = gdal.Open(input)
# Get the band
band = ds.GetRasterBand(1)

# Create the output shapefile
driver = ogr.GetDriverByName("GPKG")
out_ds = driver.CreateDataSource(output)
out_layer = out_ds.CreateLayer(output, geom_type=ogr.wkbPolygon)

# Add a field to the layer to store the pixel values
field_defn = ogr.FieldDefn("Pix_Value", ogr.OFTInteger)
out_layer.CreateField(field_defn)

# Polygonize the PNG file
gdal.Polygonize(band, None, out_layer, 0, [], callback=None)

# Close the input and output files
out_ds = None
ds = None

For the builing detection case we need to only keep the vectors with pixel value 255. Easiest solution is to use: Extract by attribute. The Python solution with GDAL can be found below.

In [ ]:
# ogr2ogr -where ID="1" outfile.gpkg infile.

# RUN from the command line inside Ubuntu
!ogr2ogr -where Pix_Value="255" bergen_-5943_1104B.gpkg bergen_-5943_1104.gpkg